In [ ]:
import json
import os
os.chdir('..')
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from pathlib import Path
import hashlib

In [2]:
MIN_COUNT = 10

In [3]:
def get_hash(str_value):
    hash_object = hashlib.sha1(str_value.encode('utf-8'))
    hex_dig = hash_object.hexdigest()
    return hex_dig

def get_file_name(url):
    file_format = Path(url).suffix
    file_name = get_hash(url) + file_format
    return file_name

In [6]:
with open('data/raw/bhajans_info.json', mode ='r') as file:
    d = json.loads(file.read())
    df = pd.DataFrame(d)
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop=True)
    df = df[df['link'].str.contains('.mp3')]

In [13]:
unwanted_text = ['Major (', 'Minor (', '~', ')']
new_text = len(unwanted_text) * [""]

for item in zip(unwanted_text, new_text):
    df['raaga'] = df['raaga'].apply(lambda x: x.replace(*item).strip())
    
final_raagas = {raaga: count for raaga, count in Counter(df['raaga']).items() if count > MIN_COUNT}

In [17]:
df = df[df['raaga'].isin(final_raagas.keys())]
df['file_name'] = ["data/raw/bhajans_audio/" + get_file_name(link) for link in df['link']]
df.reset_index(drop=True, inplace=True)
df.to_csv("data/processed/bhajans_info_cleaned.csv", index=False)

C:\Users\ak692\Anaconda3\envs\raaga\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
